# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Vector DB - BigQuery using Custom Embedding model.

* This notebook explains how to use BigQuery as a Vector database using Custom Embedding model.
* Reference
  * https://cloud.google.com/bigquery/docs/vector-search-intro?hl=ko
  * https://python.langchain.com/docs/integrations/vectorstores/google_bigquery_vector_search
  * https://api.python.langchain.com/en/stable/embeddings/langchain_core.embeddings.Embeddings.html#langchain_core.embeddings.Embeddings



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 10.1 MB/s eta 0:00:00


In [14]:
%pip install --upgrade --quiet langchain langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00


In [3]:
%pip install --upgrade --quiet sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.0 MB/s eta 0:00:00


In [5]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [6]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.



In [7]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [8]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [9]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Custom Embedding
* To use Langchain function for custom embedding, you need to implement Ebeddings interface like the following steps.

* Refer to Ebeddings interface implementation : https://api.python.langchain.com/en/stable/embeddings/langchain_core.embeddings.Embeddings.html#langchain_core.embeddings.Embeddings

* Need to implement the following two functions
  * abstract embed_documents(texts: List[str]) → List[List[float]]
  * embed_query(text: str) → List[float]

In [10]:
from langchain_core.embeddings import Embeddings
from typing import List

from sentence_transformers import SentenceTransformer

class Custom_Embedding(Embeddings):

  model = None

  def __init__(self, model_name: str):
    self.model = SentenceTransformer(model_name)

  def embed_documents(self, texts: List[str]) -> List[List[float]]:
    embeddings = self.model.encode(texts)
    return embeddings.tolist()

  def embed_query(self, text: str) -> List[float]:
    embeddings = self.model.encode([text])
    return embeddings.tolist()[0]

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Custom Embeddings model
* Use a model that was verified in Hugging face
* Use your model for the specific purpose if you have your model.

  * https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
  * https://huggingface.co/sentence-transformers/stsb-xlm-r-multilingual

In [11]:
EBEDDING_MODEL = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
embedding = Custom_Embedding(EBEDDING_MODEL)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Create dataset and table in BigQuery

In [23]:
from google.cloud import bigquery

DATASET = "vector_db_custom"
TABLE = "vector_table_custom"

client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)


Dataset(DatasetReference('ai-hangsik', 'vector_db_custom'))

## Vector Search environment

In [24]:

from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

table = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,

    #https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.utils.DistanceStrategy.html#langchain_community.vectorstores.utils.DistanceStrategy
    distance_strategy=DistanceStrategy.COSINE

)

## Read the text data that will be vectorized.

In [25]:
import pandas as pd

file_path = '/content/drive/MyDrive/projects/google_gen_ai_sample/contents/text/legal_terminology.csv'

legal_terms = pd.read_csv(file_path,sep=",", encoding='utf-8-sig')
legal_terms

,용어명,설명
0,가등기,"부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표..."
1,가등기권리자,가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임...
2,가등기권자,가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임...
3,매도인,"매도인이란 물건을 파는 사람이란 뜻으로, 부동산 임대차상에서는 건물이나 집 등을 판..."
4,가압류,가압류란 돈을 갚아야 하는 사람이 재산을 숨기거나 팔아 버릴 경우 돈을 받지 못할 ...
...,...,...
1018,특수지역권,특수지역권이란 어느 지역의 사람들이 집합체의 관계로 각자가 어떤 토지에서 초목과 야...
1019,필요비,"필요비란 임차인이 수도 설비나 전기 시설, 보일러 설치 등과 같이 임차물의 보존을 ..."
1020,하자없는 점유,하자없는 점유란 점유자가 점유하고 있는 물건의 권리자가 있는 것을 몰랐거나 알 수 ...
1021,하자있는 점유,"하자있는 점유란 점유자가 점유하고 있는 물건의 권리자가 있는 것을 알면서도, 그 점..."


## Embedding and store the vector data into BigQuery

In [26]:
import json

all_texts = legal_terms['설명'].to_list()
metadatas = [ {'Term': row['용어명'] } for idx, row in legal_terms.iterrows()]
table.add_texts(all_texts, metadatas=metadatas)
# table.add_texts(all_texts)


['385ec344e9e44007ac71b7f30589cb32',
 '33421b87457c48eb8c6cea27eee4fd07',
 '804ed8ee7e894853b33b2fb61695fe26',
 '2152c9e802ed42489210fabb89ba492d',
 'b551867831f041ba8819e3466b5a3dfd',
 '15f460d599344def8f8586a004cddc7d',
 '2703f4801294421b9122ab592d9915ea',
 '6b99b731498643799c6bfbd8a0b3f11c',
 '6a1f3eda63024c409dcb258e1af540af',
 '24e483fd99374e9aaa6804eacf457d6a',
 'b7cfd77a1a97467c83e6da7174810a20',
 '30c6d721cf7d4be0a5efeaa2f038ca95',
 '3d81e87e21304dbbbb521368a0cc0ee3',
 '017cb57e0696427e85b47d7a9d03d314',
 '29280a51bfae4320b0a9babd891f6595',
 'f280b4d467ba41ecadfd239e208c53d6',
 'b1e0e0b2982b47629e22c150429f5b7c',
 '57fa5297dacb419cb4b35828ea06e4ca',
 '5f9b0b3977564c8bbab9a78e61dd5aca',
 'e258cc7ebb684d5c96ad7df851c34003',
 '6fb4338d56be44808db96d1b240886e7',
 '9c05669d64e94f91b7472cf1d2b89fc5',
 '129e0870343d4ecbbf7996db655137aa',
 '35e71e3e1ee54f0986f07b678765bd95',
 '22de9e0f7bd141cab3c694aae143781c',
 'de293c5f8b6a4f1a981163272333ba08',
 'fee47050f49f48da88f215ad5aad9a25',
 

## Vector Search
### Search with string

In [35]:
import time
s = time.time()
query = "가등기권리자란?"

docs = table.similarity_search(query, k=5)
for doc in docs:
  print(f" {doc.metadata['Term']} - {doc.page_content}" )

e = time.time() - s
print(e)

 가등기권리자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 가등기권자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 등기청구권 - 등기청구권이란 실체권리관계에 부합하는 진실한 권리를 가지고 있는 자가 현재의 등기명의인에게 등기의 변경을 청구할 수 있는 권리를 말합니다.
 본등기 - 본등기란 임시등기인 가등기와 다르게 권리 관계의 효력을 완전하게 발생시키는 본래 등기를 의미합니다.
 가등기 - 부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표시되기 때문에 제3자에게도 대항할 수 있게 됩니다. 이 때, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 가등기라 합니다.
2.155437469482422


### Search with vector
* https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.similarity_search

In [34]:
query_vector = embedding.embed_query(query)
docs = table.similarity_search_by_vector(query_vector, k=5)
for doc in docs:
  print(f" {doc.metadata['Term']} - {doc.page_content}" )

 가등기권리자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 가등기권자 - 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
 등기청구권 - 등기청구권이란 실체권리관계에 부합하는 진실한 권리를 가지고 있는 자가 현재의 등기명의인에게 등기의 변경을 청구할 수 있는 권리를 말합니다.
 본등기 - 본등기란 임시등기인 가등기와 다르게 권리 관계의 효력을 완전하게 발생시키는 본래 등기를 의미합니다.
 가등기 - 부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표시되기 때문에 제3자에게도 대항할 수 있게 됩니다. 이 때, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 가등기라 합니다.


### Search with relevant score

In [32]:
tuples = table.similarity_search_with_relevance_scores(query, k=5)
context ={}

for tp in tuples:
    context[tp[1]] = tp[0].metadata["Term"] + " : " + tp[0].page_content
context

{0.7466386158079085: '가등기권자 : 가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.',
 0.7129046932062308: '등기청구권 : 등기청구권이란 실체권리관계에 부합하는 진실한 권리를 가지고 있는 자가 현재의 등기명의인에게 등기의 변경을 청구할 수 있는 권리를 말합니다.',
 0.709841170445082: '본등기 : 본등기란 임시등기인 가등기와 다르게 권리 관계의 효력을 완전하게 발생시키는 본래 등기를 의미합니다.',
 0.7031401704510633: '가등기 : 부동산의 권리관계는 등기를 통하여 표시하고, 이와 같이 등기된 권리관계는 외부에 표시되기 때문에 제3자에게도 대항할 수 있게 됩니다. 이 때, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 가등기라 합니다.'}

### Max marginal relavant search
* https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.max_marginal_relevance_search

In [20]:
docs = table.max_marginal_relevance_search(query= query,
                                           k=5,
                                           fetch_k = 30,
                                           lambda_mult = 0.5,
                                           brute_force = True
                                           )
for doc in docs:
  print(doc.page_content)

가등기권리자란 건물의 소유자에게 돈을 빌려 주거나 건물의 매매를 약속 하기 위해 임시등기인 가등기를 설정하는 사람을 의미합니다.
중복등기란 하나의 부동산에 보존등기가 이중으로 된 것입니다.
점유의 소란 점유보호청구권에 의하여 제기된 소입니다.
상속등기란 소유권·지상권 등 부동산등기에 의하여 공시되는 권리가 상속으로 인하여 피상속인으로부터 상속인으로 이전하였다는 것을 표시하는 등기를 말합니다.
본등기란 임시등기인 가등기와 다르게 권리 관계의 효력을 완전하게 발생시키는 본래 등기를 의미합니다.
